## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [1]:
## 
"""
(optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度
確認硬體資源
"""
!nvidia-smi

Thu Mar 19 10:45:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.50       Driver Version: 442.50       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro RTX 5000    WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   40C    P8     5W /  N/A |   1295MiB / 16384MiB |      7%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import os
import keras

# 啟用 GPU, 將 GPU 設定在有空檔的 GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # 設定 smi 看到的 GPU ID 符合實際設定的順序 (若一台機器有多張不同型號的卡時，會有此類問題)
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

Using TensorFlow backend.


In [3]:
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
"""
請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
"""
def preproc_x(x, flatten=True):
    x = 2*(((x - x.min()) / (x.max() - x.min()))-0.5)
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [6]:
"""Code Here
請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128, 128, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

model = build_mlp(input_shape=x_train.shape[1:])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
hidden_layer4 (Dense)        (None, 128)               16512     
_________________________________________________________________
hidden_layer5 (Dense)        (None, 128)               16512     
_________________________________________________________________
output (Dense)               (None, 10)                1290

In [7]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [8]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [9]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 4s 86us/step - loss: 1.7078 - accuracy: 0.3941 - val_loss: 1.5509 - val_accuracy: 0.4475
Epoch 2/100
50000/50000 [==============================] - 4s 82us/step - loss: 1.4723 - accuracy: 0.4802 - val_loss: 1.4722 - val_accuracy: 0.4770
Epoch 3/100
50000/50000 [==============================] - 4s 82us/step - loss: 1.3568 - accuracy: 0.5197 - val_loss: 1.3940 - val_accuracy: 0.5015
Epoch 4/100
50000/50000 [==============================] - 4s 86us/step - loss: 1.2589 - accuracy: 0.5562 - val_loss: 1.3880 - val_accuracy: 0.5135
Epoch 5/100
50000/50000 [==============================] - 5s 100us/step - loss: 1.1826 - accuracy: 0.5813 - val_loss: 1.3755 - val_accuracy: 0.5226
Epoch 6/100
50000/50000 [==============================] - 5s 96us/step - loss: 1.1047 - accuracy: 0.6082 - val_loss: 1.3949 - val_accuracy: 0.5141
Epoch 7/100
50000/50000 [==============================] - 5s

50000/50000 [==============================] - 5s 97us/step - loss: 0.1094 - accuracy: 0.9641 - val_loss: 3.9140 - val_accuracy: 0.5297
Epoch 56/100
50000/50000 [==============================] - 5s 97us/step - loss: 0.1129 - accuracy: 0.9621 - val_loss: 4.0254 - val_accuracy: 0.5227
Epoch 57/100
50000/50000 [==============================] - 5s 97us/step - loss: 0.1006 - accuracy: 0.9658 - val_loss: 4.1633 - val_accuracy: 0.5213
Epoch 58/100
50000/50000 [==============================] - 5s 98us/step - loss: 0.1218 - accuracy: 0.9598 - val_loss: 3.8890 - val_accuracy: 0.5136
Epoch 59/100
50000/50000 [==============================] - 5s 98us/step - loss: 0.1164 - accuracy: 0.9618 - val_loss: 3.8481 - val_accuracy: 0.5191
Epoch 60/100
50000/50000 [==============================] - 5s 98us/step - loss: 0.0909 - accuracy: 0.9701 - val_loss: 4.1772 - val_accuracy: 0.5219
Epoch 61/100
50000/50000 [==============================] - 5s 97us/step - loss: 0.1038 - accuracy: 0.9664 - val_loss: 

In [ ]:
import matplotlib.pyplot as plt


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()